In [18]:
""" Indentifies nodes in the the connectivity matrixes from MRI that are not connected, and removes those nodes from the matrixes"""

import os
import glob
import numpy as np
import networkx as nx
import warnings

def load_and_filter_matrices(directory_path, output_dir, identifiers):
    # Only load specific .csv file types
    csv_files = glob.glob(os.path.join(directory_path, '**/*tckcount_SIFT2.csv'), recursive=True)
    matrix_lists = {identifier: [] for identifier in identifiers}
    disconnected_nodes_dict = {identifier: set() for identifier in identifiers}  # Change to a dictionary of sets
    
    for file_path in csv_files:
        for identifier in identifiers:
            if identifier in os.path.dirname(file_path):
                # Specify the correct delimiter
                matrix = np.loadtxt(file_path, dtype=float, delimiter=' ')
                G = nx.from_numpy_array(matrix)
                if not nx.is_connected(G):
                    warnings.warn(f"The graph in {file_path} from directory {identifier} was not connected before thresholding.")
                    largest_cc = max(nx.connected_components(G), key=len)
                    not_connected_nodes = [node for node in G.nodes if node not in largest_cc]
                    graph_name = os.path.basename(file_path)
                    disconnected_nodes_dict[identifier].update(not_connected_nodes)  # Add the nodes to the set
                    
                    # Save the disconnected nodes to a separate file only if there are any
                    if not_connected_nodes:
                        # Increase node number by 1
                        not_connected_nodes = [node+1 for node in not_connected_nodes]
                        # Include the subject name in the name of the disconnected nodes text file
                        subject_name = os.path.dirname(file_path).split('/')[-2]
                        disconnected_nodes_file = os.path.join(output_dir, f'disconnected_nodes_{identifier}_{subject_name}_{graph_name}.txt')
                        np.savetxt(disconnected_nodes_file, not_connected_nodes, fmt='%d')
                    
                matrix_lists[identifier].append((file_path, matrix))
    
    for identifier, matrices in matrix_lists.items():
        if matrices:
            # Create a new directory for each identifier
            identifier_dir = os.path.join(output_dir, identifier)
            os.makedirs(identifier_dir, exist_ok=True)
            for file_path, matrix in matrices:
                # Remove the corresponding rows and columns from the matrix
                matrix = np.delete(matrix, list(disconnected_nodes_dict[identifier]), axis=0)
                matrix = np.delete(matrix, list(disconnected_nodes_dict[identifier]), axis=1)
                # Save the filtered matrices in the newly created directories
                subject_name = os.path.dirname(file_path).split('/')[-2]
                atlas_name = os.path.basename(file_path).split('_')[0]
                study_name = file_path.split('/')[-4]
                study_dir = os.path.join(identifier_dir, study_name)
                os.makedirs(study_dir, exist_ok=True)
                output_file_path = os.path.join(study_dir, f'filtered_{subject_name}_{atlas_name}.csv')
                np.savetxt(output_file_path, matrix, delimiter=',')
   

In [20]:
"""Threshold all filtered connectivity matrices to the highest minimal edge density across all the graphs"""

import os
import glob
import numpy as np
import networkx as nx
import warnings

def threshold_network(directory_path, output_dir):
    csv_files = glob.glob(os.path.join(directory_path, '**', '*filtered*connectome*'), recursive=True)
    min_edge_density = 0
    individual_min_edge_densities = []  # List to store individual minimum edge densities

    # First pass: Determine the minimal edge density to remain connected across all graphs
    for file_path in csv_files:
        matrix = np.loadtxt(file_path, dtype=float, delimiter=',')
        G = nx.from_numpy_array(matrix)
        
        atlas_output_dir = os.path.join(output_dir, os.path.basename(os.path.dirname(file_path)))
        os.makedirs(atlas_output_dir, exist_ok=True)
        
        if not nx.is_connected(G):
            warnings.warn(f"The graph in {file_path} was not connected before thresholding.")
            largest_cc = max(nx.connected_components(G), key=len)
            not_connected_nodes = [node for node in G.nodes if node not in largest_cc]
            
            unconnected_file_path = os.path.join(atlas_output_dir, os.path.basename(file_path).replace('.csv', '_INITIAL_UNconnected_nodes.txt'))
            np.savetxt(unconnected_file_path, not_connected_nodes, fmt='%d')
            
        threshold = 0
        while nx.is_connected(G):
            threshold += 1
            matrix[matrix < threshold] = 0
            G = nx.from_numpy_array(matrix)

            if not nx.is_connected(G):
                break

        threshold -= 1
        edge_density = nx.density(G)
        min_edge_density = max(min_edge_density, edge_density) #this needs to be max to avoid disconnection

        # Save individual minimum edge density and corresponding graph name
        individual_min_edge_densities.append([os.path.basename(file_path), edge_density])

    # Save the final min_edge_density to a separate CSV file
    directory_name = os.path.basename(directory_path)
    np.savetxt(os.path.join(output_dir, f"{directory_name}_min_edge_density.csv"), [min_edge_density], delimiter=',')

    # Save individual minimum edge densities to a separate CSV file
    np.savetxt(os.path.join(output_dir, f"{directory_name}_individual_min_edge_densities.csv"), individual_min_edge_densities, delimiter=',', fmt='%s')


    # Second pass: Threshold all the graphs to the highest minimal edge density across all the graphs
    for file_path in csv_files:
        matrix = np.loadtxt(file_path, dtype=float, delimiter=',')
        G = nx.from_numpy_array(matrix)
        
        atlas_name = os.path.basename(os.path.dirname(file_path))
        atlas_output_dir = os.path.join(output_dir, atlas_name)
        os.makedirs(atlas_output_dir, exist_ok=True)

        
        threshold = 0
        while nx.is_connected(G) and nx.density(G) > (1.05 * min_edge_density): # this needs to be larger than 1 to avoid disconnection
            threshold += 1
            matrix[matrix < threshold] = 0
            temp_G = nx.from_numpy_array(matrix)

            if not nx.is_connected(temp_G):
                print(f"Error: The graph in {file_path} became disconnected when thresholded to edge density {nx.density(G)}.")
                break

            G = temp_G
    

         # Save the thresholded matrix to a new CSV file in the output directory
        SCM_filename = os.path.splitext(os.path.basename(file_path))[0] + "_threshold_SCM.csv"
        np.savetxt(os.path.join(atlas_output_dir, SCM_filename), matrix, delimiter=',')

    

In [24]:
"""Create an averaged connectivity matrix across subjects"""
def average_matrices(path, averaged_data_path):
    files = glob.glob(os.path.join(path, '**/*SCM.csv'), recursive=True)
    for subfolder in os.listdir(path):
        subfolder_path = os.path.join(path, subfolder)
        if os.path.isdir(subfolder_path):
            matrix_list = []
            for file_path in files:
                if subfolder in os.path.dirname(file_path):
                    matrix = np.loadtxt(file_path, dtype=float, delimiter=',')
                    # Add this line after reading the matrix from the file
                    if matrix.shape == (445, 445):
                        print(f"The matrix in {file_path} has the dimensions 445 x 445")
                    matrix_list.append(matrix)
            if matrix_list:
                averaged_matrix = np.mean(matrix_list, axis=0)
                averaged_data_subfolder_path = os.path.join(averaged_data_path, subfolder)
                os.makedirs(averaged_data_subfolder_path, exist_ok=True)
                np.savetxt(os.path.join(averaged_data_subfolder_path, f'averaged_Schaefer_{subfolder}_SCM.csv'), 
                           averaged_matrix,
                           delimiter=',')

In [26]:
"""saves edge density of all averaged SCM matrices to a file"""

def check_edge_density(directory_path):
    # Get all .csv files in the directory and its subdirectories
    csv_files = glob.glob(os.path.join(directory_path, '**/*.csv'), recursive=True)

    # Define the output file path
    output_file = os.path.join(directory_path, 'Averaged_edge_densities.txt')

    with open(output_file, 'w') as f:
        for file_path in csv_files:
            # Load the matrix
            matrix = np.loadtxt(file_path, dtype=float, delimiter=',')
            
            # Create a graph from the matrix
            G = nx.from_numpy_array(matrix)
            
            # Calculate the edge density
            edge_density = nx.density(G)
            
            # Write the file path and edge density to the output file
            f.write(f"{file_path}: {edge_density}\n")



In [19]:
""" Average the individual thresholded matrices"""
import numpy as np

def average_matrices(path, averaged_data_path, *identifiers): #specify the number of node in each parcellation 
    files = glob.glob(os.path.join(path, '**/*SCM.csv'), recursive=True)
    
    matrix_lists = {identifier: [] for identifier in identifiers}
    averaged_data_paths = {identifier: os.path.join(averaged_data_path, identifier) for identifier in identifiers}
    
    for file_path in files:
        for identifier in identifiers:
            if identifier in os.path.dirname(file_path):
                matrix = np.loadtxt(file_path, dtype=float, delimiter=',')
                matrix_lists[identifier].append(matrix)

    for identifier in identifiers:
        os.makedirs(averaged_data_paths[identifier], exist_ok=True)
        if matrix_lists[identifier]:
            averaged_matrix = sum(matrix_lists[identifier]) / len(matrix_lists[identifier])
            np.savetxt(os.path.join(averaged_data_paths[identifier], f'averaged_Schaefer_{identifier}_SCM.csv'), 
                       averaged_matrix,
                       delimiter=',')